In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [5]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 17.423269
Training accuracy: 8.4%
Validation accuracy: 10.3%
Loss at step 100: 2.260211
Training accuracy: 72.0%
Validation accuracy: 71.0%
Loss at step 200: 1.800829
Training accuracy: 75.4%
Validation accuracy: 73.4%
Loss at step 300: 1.564327
Training accuracy: 76.8%
Validation accuracy: 74.3%
Loss at step 400: 1.407341
Training accuracy: 77.6%
Validation accuracy: 74.6%
Loss at step 500: 1.290650
Training accuracy: 78.4%
Validation accuracy: 74.7%
Loss at step 600: 1.198971
Training accuracy: 79.1%
Validation accuracy: 74.8%
Loss at step 700: 1.124419
Training accuracy: 79.3%
Validation accuracy: 74.8%
Loss at step 800: 1.062310
Training accuracy: 79.7%
Validation accuracy: 75.0%
Test accuracy: 82.5%


In [6]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [7]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 22.141140
Minibatch accuracy: 4.7%
Validation accuracy: 7.6%
Minibatch loss at step 500: 1.095135
Minibatch accuracy: 82.0%
Validation accuracy: 76.2%
Minibatch loss at step 1000: 1.009715
Minibatch accuracy: 79.7%
Validation accuracy: 77.2%
Minibatch loss at step 1500: 1.523743
Minibatch accuracy: 73.4%
Validation accuracy: 77.8%
Minibatch loss at step 2000: 1.026660
Minibatch accuracy: 78.1%
Validation accuracy: 78.2%
Minibatch loss at step 2500: 0.840471
Minibatch accuracy: 81.2%
Validation accuracy: 78.6%
Minibatch loss at step 3000: 0.903822
Minibatch accuracy: 78.9%
Validation accuracy: 78.7%
Test accuracy: 85.3%


In [8]:
batch_size = 128
hnnum = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hnnum]))
  biases_1 = tf.Variable(tf.zeros([hnnum]))
  weights_2 = tf.Variable(
    tf.truncated_normal([hnnum, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  reluout = tf.nn.relu(logits_1)
  logits_2 = tf.matmul(reluout, weights_2) + biases_2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits_2, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_2)
  vlogits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
  vreluout = tf.nn.relu(vlogits_1)
  vlogits_2 = tf.matmul(vreluout, weights_2) + biases_2
  valid_prediction = tf.nn.softmax(vlogits_2)
  tlogits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
  treluout = tf.nn.relu(tlogits_1)
  tlogits_2 = tf.matmul(treluout, weights_2) + biases_2
  test_prediction = tf.nn.softmax(tlogits_2)

In [9]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 302.013672
Minibatch accuracy: 3.9%
Validation accuracy: 26.0%
Minibatch loss at step 500: 8.152968
Minibatch accuracy: 88.3%
Validation accuracy: 81.4%
Minibatch loss at step 1000: 6.782022
Minibatch accuracy: 84.4%
Validation accuracy: 81.5%
Minibatch loss at step 1500: 15.634326
Minibatch accuracy: 71.9%
Validation accuracy: 78.8%
Minibatch loss at step 2000: 2.606236
Minibatch accuracy: 87.5%
Validation accuracy: 81.2%
Minibatch loss at step 2500: 6.865630
Minibatch accuracy: 85.2%
Validation accuracy: 82.5%
Minibatch loss at step 3000: 4.717116
Minibatch accuracy: 84.4%
Validation accuracy: 82.2%
Test accuracy: 88.6%


In [10]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 286.824860
Minibatch accuracy: 12.5%
Validation accuracy: 34.6%
Minibatch loss at step 500: 6.363008
Minibatch accuracy: 85.9%
Validation accuracy: 80.8%
Minibatch loss at step 1000: 6.941345
Minibatch accuracy: 85.9%
Validation accuracy: 81.4%
Minibatch loss at step 1500: 12.453547
Minibatch accuracy: 78.9%
Validation accuracy: 80.9%
Minibatch loss at step 2000: 3.831989
Minibatch accuracy: 84.4%
Validation accuracy: 81.5%
Minibatch loss at step 2500: 5.140130
Minibatch accuracy: 80.5%
Validation accuracy: 81.7%
Minibatch loss at step 3000: 8.375126
Minibatch accuracy: 78.1%
Validation accuracy: 81.8%
Test accuracy: 88.6%


In [11]:
batch_size = 128
hnnum = 2048

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hnnum]))
  biases_1 = tf.Variable(tf.zeros([hnnum]))
  weights_2 = tf.Variable(
    tf.truncated_normal([hnnum, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  reluout = tf.nn.relu(logits_1)
  logits_2 = tf.matmul(reluout, weights_2) + biases_2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits_2, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_2)
  vlogits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
  vreluout = tf.nn.relu(vlogits_1)
  vlogits_2 = tf.matmul(vreluout, weights_2) + biases_2
  valid_prediction = tf.nn.softmax(vlogits_2)
  tlogits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
  treluout = tf.nn.relu(tlogits_1)
  tlogits_2 = tf.matmul(treluout, weights_2) + biases_2
  test_prediction = tf.nn.softmax(tlogits_2)

In [12]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 582.840210
Minibatch accuracy: 4.7%
Validation accuracy: 37.7%
Minibatch loss at step 500: 16.073425
Minibatch accuracy: 91.4%
Validation accuracy: 82.2%
Minibatch loss at step 1000: 13.106039
Minibatch accuracy: 87.5%
Validation accuracy: 83.0%
Minibatch loss at step 1500: 22.678547
Minibatch accuracy: 80.5%
Validation accuracy: 81.3%
Minibatch loss at step 2000: 8.699547
Minibatch accuracy: 85.9%
Validation accuracy: 83.6%
Minibatch loss at step 2500: 13.933823
Minibatch accuracy: 79.7%
Validation accuracy: 83.4%
Minibatch loss at step 3000: 5.897406
Minibatch accuracy: 85.2%
Validation accuracy: 82.8%
Test accuracy: 89.4%


In [13]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 526.927917
Minibatch accuracy: 21.1%
Validation accuracy: 37.1%
Minibatch loss at step 500: 25.815725
Minibatch accuracy: 88.3%
Validation accuracy: 82.0%
Minibatch loss at step 1000: 14.166840
Minibatch accuracy: 87.5%
Validation accuracy: 81.0%
Minibatch loss at step 1500: 26.627745
Minibatch accuracy: 79.7%
Validation accuracy: 81.7%
Minibatch loss at step 2000: 4.949147
Minibatch accuracy: 85.9%
Validation accuracy: 83.5%
Minibatch loss at step 2500: 10.135880
Minibatch accuracy: 82.8%
Validation accuracy: 82.2%
Minibatch loss at step 3000: 5.564411
Minibatch accuracy: 85.2%
Validation accuracy: 83.5%
Minibatch loss at step 3500: 3.676456
Minibatch accuracy: 89.8%
Validation accuracy: 84.2%
Minibatch loss at step 4000: 8.760988
Minibatch accuracy: 85.2%
Validation accuracy: 84.4%
Minibatch loss at step 4500: 5.873106
Minibatch accuracy: 82.8%
Validation accuracy: 84.4%
Minibatch loss at step 5000: 2.191010
Minibatch accuracy: 89.8%
Validation a